## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from vacation_search_config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Ushuaia,AR,2022-01-28 02:51:13,-54.8000,-68.3000,49.66,81,75,4.61
1,1,Caravelas,BR,2022-01-28 02:51:13,-17.7125,-39.2481,75.29,91,3,5.26
2,2,Atar,MR,2022-01-28 02:51:13,20.5169,-13.0499,63.97,38,100,6.58
3,3,Bredasdorp,ZA,2022-01-28 02:51:13,-34.5322,20.0403,60.12,82,1,2.30
4,4,Luanda,AO,2022-01-28 02:51:14,-8.8368,13.2343,77.07,84,100,3.27


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip?"))
max_temp = float(input("What is your derired maximum temperature for your trip?"))

What is your desired minimum temperature for your trip?30
What is your derired maximum temperature for your trip?50


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                        (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Ushuaia,AR,2022-01-28 02:51:13,-54.8000,-68.3000,49.66,81,75,4.61
10,10,Great Bend,US,2022-01-28 02:47:59,38.3645,-98.7648,33.62,81,0,8.05
13,13,Grenada,ES,2022-01-28 02:50:41,37.1882,-3.6067,48.18,77,0,4.61
30,30,Bjaeverskov,DK,2022-01-28 02:51:19,55.4576,12.0365,41.68,75,11,17.27
33,33,Kruszwica,PL,2022-01-28 02:51:19,52.6756,18.3313,38.32,100,95,26.53
...,...,...,...,...,...,...,...,...,...,...
674,674,Berezovka,BY,2022-01-28 03:07:48,53.7214,25.4993,33.93,98,100,15.77
678,678,Kalaleh,IR,2022-01-28 03:07:49,37.3807,55.4916,44.28,95,100,6.98
682,682,Arcata,US,2022-01-28 03:07:50,40.8665,-124.0828,47.59,83,0,3.44
684,684,Muborak,UZ,2022-01-28 03:07:51,39.2553,65.1528,48.49,37,100,8.10


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Ushuaia,AR,2022-01-28 02:51:13,-54.8000,-68.3000,49.66,81,75,4.61
10,10,Great Bend,US,2022-01-28 02:47:59,38.3645,-98.7648,33.62,81,0,8.05
13,13,Grenada,ES,2022-01-28 02:50:41,37.1882,-3.6067,48.18,77,0,4.61
30,30,Bjaeverskov,DK,2022-01-28 02:51:19,55.4576,12.0365,41.68,75,11,17.27
33,33,Kruszwica,PL,2022-01-28 02:51:19,52.6756,18.3313,38.32,100,95,26.53
...,...,...,...,...,...,...,...,...,...,...
674,674,Berezovka,BY,2022-01-28 03:07:48,53.7214,25.4993,33.93,98,100,15.77
678,678,Kalaleh,IR,2022-01-28 03:07:49,37.3807,55.4916,44.28,95,100,6.98
682,682,Arcata,US,2022-01-28 03:07:50,40.8665,-124.0828,47.59,83,0,3.44
684,684,Muborak,UZ,2022-01-28 03:07:51,39.2553,65.1528,48.49,37,100,8.10


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[['City', 'Country', 'Max Temp', 'Humidity', 'Cloudiness', 'Lat', 'Lng']].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Humidity,Cloudiness,Lat,Lng,Hotel Name
0,Ushuaia,AR,49.66,81,75,-54.8000,-68.3000,
10,Great Bend,US,33.62,81,0,38.3645,-98.7648,
13,Grenada,ES,48.18,77,0,37.1882,-3.6067,
30,Bjaeverskov,DK,41.68,75,11,55.4576,12.0365,
33,Kruszwica,PL,38.32,100,95,52.6756,18.3313,
41,Vilanova Del Cami,ES,49.53,62,0,41.5717,1.6375,
48,Cananea,MX,49.78,25,96,30.9500,-110.3000,
50,Castlegar,CA,30.88,100,76,49.2998,-117.6689,
57,Mao,ES,45.16,100,0,39.8885,4.2658,
62,Sarakhs,IR,46.92,79,0,36.5449,61.1577,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.")
        

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df

,City,Country,Max Temp,Humidity,Cloudiness,Lat,Lng,Hotel Name
0,Ushuaia,AR,49.66,81,75,-54.8000,-68.3000,Albatross Hotel
10,Great Bend,US,33.62,81,0,38.3645,-98.7648,Best Western Angus Inn
13,Grenada,ES,48.18,77,0,37.1882,-3.6067,HOTEL REINA CRISTINA
30,Bjaeverskov,DK,41.68,75,11,55.4576,12.0365,Allan Lund Tyllesen
33,Kruszwica,PL,38.32,100,95,52.6756,18.3313,Ośrodek Szkolenia Żeglarskiego: Pole Namiotowe
...,...,...,...,...,...,...,...,...
674,Berezovka,BY,33.93,98,100,53.7214,25.4993,Nad Nemanom Usad'ba
678,Kalaleh,IR,44.28,95,100,37.3807,55.4916,عطر و ادکلن پارسیس
682,Arcata,US,47.59,83,0,40.8665,-124.0828,"Hampton Inn & Suites Arcata, CA"
684,Muborak,UZ,48.49,37,100,39.2553,65.1528,Muborak temiryoʻl vakzali


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))